A code to suggest cocktails based on ingredients

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import csv
import time
import re
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

First, we get the recipes from www.drinksmixer.com 

In [3]:
page_address = []
for i in range(1,9):
    page_address.append("http://www.drinksmixer.com/search/?q=cocktail&d=1&p=" + str(i))

list_1 = []

for i in range(8):   
    page = requests.get(page_address[i], time.sleep(.5))
    
    
#create beautiful soup object
    soup = bs(page.text, 'html.parser')


    for i in range(2):
        list_1.append(soup.find_all(class_= "l1a")[i].find_all("a"))

###flatten list_1
list_flat = [item for sublist in list_1 for item in sublist]


link_tails = []
for i in range(len(list_flat)):
        link_tails.append(list_flat[i]["href"])
        
        
### Now Build the recipe links ###

recipe_links = []
for i in range(len(link_tails)):
    recipe_links.append("http://www.drinksmixer.com" + link_tails[i]) 
    


In [6]:
recipe_links[0:10]

['http://www.drinksmixer.com/drink10391.html',
 'http://www.drinksmixer.com/drink7469.html',
 'http://www.drinksmixer.com/drink10313.html',
 'http://www.drinksmixer.com/drink8489.html',
 'http://www.drinksmixer.com/drink688.html',
 'http://www.drinksmixer.com/drink694.html',
 'http://www.drinksmixer.com/drink3411.html',
 'http://www.drinksmixer.com/drink740.html',
 'http://www.drinksmixer.com/drink746.html',
 'http://www.drinksmixer.com/drink750.html']

Now we get the recipes

In [ ]:
cocktail_rec = []
for i in range(len(recipe_links)):
    
    page = requests.get(recipe_links[i], time.sleep(.5))

    soup = bs(page.text, 'html.parser')

    cocktail_name = soup.find(class_ = "fn recipe_title").string


    ing_name = soup.find(class_ = "ingredients").find_all(class_="name")
    ing_amount = soup.find(class_ = "ingredients").find_all(class_="amount")

    ingredients = [item.string for item in ing_name]

    cocktail_el = [cocktail_name, ingredients]
    cocktail_rec.append(cocktail_el)

In [ ]:
###  connect the words in ingredients
    
for j in range(len(cocktail_rec)):
    for i in range(len(cocktail_rec[j][1])):
        cocktail_rec[j][1][i] = cocktail_rec[j][1][i].replace(" ","")

In [ ]:
###  creat the recipe list
rec_list=[]
for i in range(len(cocktail_rec)):
    rec=""
    for word in cocktail_rec[i][1]:
        rec = rec + word + " "
    list_elem = [cocktail_rec[i][0].replace(" Cocktail","").replace(" recipe", "") , rec]
    rec_list.append(list_elem)

In [8]:
###  make a dataframe and save as cocktail_rec

recipe_df = pd.DataFrame(rec_list , columns = ["Cocktail" , "Recipe"])

recipe_df.to_csv(r'E:\...\cocktail_rec.csv', index=False)

Now we build the recommender system

In [9]:
###  Read the recipe dataframe

recipe_df = pd.read_csv('E:\...\cocktail_rec.csv')

###  Find the count matrix

count = CountVectorizer() 
count_matrix = count.fit_transform(recipe_df["Recipe"])

### generating the cosine similarity matrix

cos_sim = cosine_similarity(count_matrix, count_matrix)

cos_sim_df = pd.DataFrame(cos_sim )


In [10]:
###  Build a function that gives the n best matches to a chosen cocktail

def recommender(cocktail_name):
    coc = cocktail_name
    ind  = recipe_df[recipe_df["Cocktail"]==coc].index[0]

    score = pd.Series(cos_sim_df[ind]).sort_values(ascending= False)

    best_match_ind = list(score.iloc[1:6].index)



## add same scores as 5th cocktail in the list
    for i in range(5,712):
        if score.iloc[i+1]==score.iloc[i]:
            best_match_ind.append(list(score.iloc[i+1:i+2].index)[0])
        else:
            break
    
    
##make the list of at least top 5 matches     
    best_match=[]
    for i in best_match_ind:
        best_match.append(recipe_df.iloc[i][0])
        
    return print("If you are a fan of " + coc + "," + " you should also try", best_match)

An example:

In [11]:
recommender("Abbey")

If you are a fan of Abbey, you should also try ['Jewel', 'Casino', 'Bijou', 'Tuxedo', 'Tailspin']
